# OneEye: Military Airfiled Monitoring with Computer Vision
## AAI-521 Team 1 Final Project

Team 1:  Tyler Foreman

University of San Diego, Applied Artificial Intelligence

Date:  December 11, 2023

GitHub Repository: https://github.com/t4ai/one-eye-aircraft-detector

In [5]:
# import base libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import pathlib
import PIL
import PIL.Image
import xml.etree.ElementTree as ET

# import keras and supporting libraries
import keras
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from keras.datasets import mnist, cifar10
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D
from keras.layers import BatchNormalization
from keras.utils import to_categorical
from keras.layers import Dense
from keras import optimizers
from tensorflow.keras.optimizers import SGD, Adam
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K

## Exploratory Data Analysis

Scan the images and bounding box labels to understand the nature of the dataset.  Objectives are:

1.  Understand the distribution of image sizes
2.  Understand the distribution of image channels
3.  Analyze the class balances and representation across the dataset
4.  Visualize sample images with horizontal bounding boxes overlaid
5.  Visualize sample images with oriented bounding boxes overlaid

In [6]:
dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
archive = tf.keras.utils.get_file(origin=dataset_url, extract=True)
data_dir = pathlib.Path(archive).with_suffix('')
data_dir

228813984/228813984 [==============================] - 13s 0us/step


PosixPath('/Users/tylerforeman/.keras/datasets/flower_photos')

In [ ]:
data_dir = pathlib.Path(archive).with_suffix('')

In [ ]:
# helper function to read image files and extract dimensions
def analyze_image_dimensions(image_path):
    imgs = list(data_dir.glob(image_path))
    for img in imgs:
        PIL.Image.open(str(img))